In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
import random
import csv
from operator import itemgetter
from networkx.algorithms import tree
import seaborn as sns

In [ ]:
## Data Sorting ##

df = pd.read_csv('FoodWebDataBase.csv')
consumer = df['con.taxonomy']
resource = df['res.taxonomy']

allNodes = list(consumer) + list(resource)
allNodes = set(allNodes) # Removes duplicates
print(len(allNodes))
#print(allNodes)


edgelistdf = df[['con.taxonomy','res.taxonomy']]
edgelist = list(map(tuple, edgelistdf.to_numpy()))
edgelist


In [ ]:
# Function for Redundancy Calculation

from itertools import tee

def TreeRedundancy(spanningTree):

    def pairwise(iterable):
        a, b = tee(iterable)
        next(b, None)
        return zip(a, b)
    
    reversedcount = spanningTree[::-1]
    final = pairwise(reversedcount)
    SpanningTreeEdges = list(final)
    for i in SpanningTreeEdges:
        numOfEdgesInSpanningTree.append(i)
    
    return

# MAIN

In [ ]:
########## 290 loop of the foodwebs #########

AllFoodWebData = [] # Collects data of all of the foodwebs

## List of 290 foodwebs ##

foodwebList = df['foodweb.name'].unique()


############################################## Loop through the foodweb list #######################################

for i in range(len(foodwebList)):
    
    
    print(f"<<<<<<<<< Foodweb Name: {foodwebList[i]}>>>>>>>>>>>>")
    
########## Sort consumer and resource column, find all node and edge list #########
    Foodweb = df.loc[df['foodweb.name'] == foodwebList[i]]  # Retrieve each foodweb con and res

    consumer = Foodweb['con.taxonomy']
    resource = Foodweb['res.taxonomy']

    allNodes = list(consumer) + list(resource) # Combine all nodes
    allNodes = set(allNodes) # Removes duplicates
    #print(len(allNodes))


    edgelistdf = Foodweb[['con.taxonomy','res.taxonomy']]
    edgelist = list(map(tuple, edgelistdf.to_numpy()))
    #edgelist

    G = nx.Graph()  # Define type of graph (undirected)

    G.add_nodes_from(allNodes)  # Add all nodes from the foodweb
    G.add_edges_from(edgelist)  # Add edge list to the graph

    #print(nx.info(G))  # Graph info

########## Print graph (undirected) #########
    plt.figure()
    plt.subplots(figsize=(30, 30))
    plt.title(foodwebList[i], size=30)
    nx.draw_networkx(G, with_labels=True)
    plt.show
    
    
    
####### Use DFS Tree To Find List of Resource Species #########
    sortedNodes = sorted(G.degree, key = lambda x: x[1], reverse = True) # Sort node based on highest degree to lowest
    
    ## Sorting nodes in dictionary by its degree value to be used as look up ##
    degree = {}
    for i in sortedNodes:
        degree[i[0]] = i[1]
    #print(degree)
    
    ## Connects the network with the least number of links to find Resource Species  ##
    spanningtree = nx.depth_first_search.dfs_tree(G,sortedNodes[0][0])
    
    edgelistst = list(spanningtree)
    
    ## Find the resource species using degree nodes (no of node adjacent to each node) ##
    spanningTreeSortedNodes = sorted(spanningtree.degree, key = lambda x: x[1], reverse = True)
    #print(spanningTreeSortedNodes)

    resourceSpecies = []

    for i in spanningTreeSortedNodes:  
        if i[1] == 1:
            resourceSpecies.append(i)
    
    ## Obtaining the resource species and remove apex species if found it list ##
    finalResourceSpecies = []

    for i in resourceSpecies:
        if i[0]  != sortedNodes[0][0]:       
            finalResourceSpecies.append(i[0]) 
        
        
        
######## Spanning Tree Loop  #########
    SpanningTreeLists = []     # List of all individual spanning trees in the food web

    redundantLinksCount = 0
    numOfEdgesInSpanningTree = []

    for i in finalResourceSpecies:                  # Goes through each resource species   

        spanningtree = []
        predatorList = []

        #print("Level 0 (Source Species): ",i)

        if i not in spanningtree:      # Checks if root node is in the tree
            spanningtree.append(i)

        for j in edgelist:                 # If node is in edgelist connection  
            if j[1] == i:                    # Check if species is a resource species in edgelist
                if degree[j[0]] > degree[i]: # Checks if the connected node has a higher degree
                    predatorList.append(j[0]) 

        if len(predatorList) == 0:                 # If length is zero print final spanning tree sequence
            #print("Final Spanning Tree: ", spanningtree)
            SpanningTreeLists.append(spanningtree)    # Appends the final spanning tree in a list
            TreeRedundancy(spanningtree)              # Adds final spanning tree edge for redundancy count
            continue

        pred = random.choice(predatorList)    # Select a random predator going up the food chain
        spanningtree.append(pred)

        #print("level 1: ", predatorList)
    ##############################################################    
        predatorList2 = []

        for k in edgelist:                        # Loop through predator list     
            if k[1] == pred:
                if degree[k[0]] > degree[pred]:
                    predatorList2.append(k[0])

        if len(predatorList2) == 0:               # If length is zero print final spanning tree sequence
            #print("Final Spanning Tree: ", spanningtree)

            redundantLinksCount = redundantLinksCount + len(predatorList)-1 
            #print("Total Redundant links in this foodweb: ",redundantLinksCount)
            SpanningTreeLists.append(spanningtree)    # Appends the final spanning tree in a list
            TreeRedundancy(spanningtree)              # Adds final spanning tree edge for redundancy count
            continue

        pred2 = random.choice(predatorList2)    # Select a random predator going up the food chain
        spanningtree.append(pred2)



        #print("level 2: ", predatorList2)
    ##############################################################    
        predatorList3 = []

        for h in edgelist:                        # Loop through predator list     
            if h[1] == pred2:
                if degree[h[0]] > degree[pred2]:
                    predatorList3.append(h[0])

        if len(predatorList3) == 0:                # If length is zero print final spanning tree sequence
#             print("Final Spanning Tree: ", spanningtree)

            redundantLinksCount = redundantLinksCount + len(predatorList) + len(predatorList2) -2
            #print("Total Redundant links in this foodweb: ",redundantLinksCount)
            SpanningTreeLists.append(spanningtree)    # Appends the final spanning tree in a list
            TreeRedundancy(spanningtree)              # Adds final spanning tree edge for redundancy count
            continue

        pred3 = random.choice(predatorList3)    # Select a random predator going up the food chain
        spanningtree.append(pred3)



        #print("level 3: ", predatorList3)
    ##############################################################    
        predatorList4 = []

        for h in edgelist:                        # Loop through predator list     
            if h[1] == pred3:
                if degree[h[0]] > degree[pred3]:
                    predatorList3.append(h[0])

        if len(predatorList4) == 0:             # If length is zero print final spanning tree sequence
#             print("Final Spanning Tree: ", spanningtree)

            SpanningTreeLists.append(spanningtree)    # Appends the final spanning tree in a list
            TreeRedundancy(spanningtree)              # Adds final spanning tree edge for redundancy count
            continue

        pred4 = random.choice(predatorList4)    # Select a random predator going up the food chain
        spanningtree.append(pred4)    


        #print("level 4: ", predatorList4)
    ##############################################################    
        predatorList5 = []

        for h in edgelist:                        # Loop through predator list     
            if h[1] == pred4:
                if degree[h[0]] > degree[pred4]:
                    predatorList4.append(h[0])

        if len(predatorList5) == 0:             # If length is zero print final spanning tree sequence
#             print("Final Spanning Tree: ", spanningtree)

            SpanningTreeLists.append(spanningtree)    # Appends the final spanning tree in a list
            TreeRedundancy(spanningtree)              # Adds final spanning tree edge for redundancy count
            continue

        pred5 = random.choice(predatorList5)    # Select a random predator going up the food chain
        spanningtree.append(pred5)    

        #print("level 5: ", predatorList5)
    spantreelinks = set(numOfEdgesInSpanningTree) # to remove same edge in spanning tree list
    redundantLinks = len(edgelist) - len(spantreelinks)

    print("Number of nodes in the food web: ", len(allNodes))
    print("Number of Original of edges: ",len(edgelist))
    print("Number of edges in spanning tree: ",len(spantreelinks))
    print("Total Redundant links in this foodweb: ",redundantLinks)
    print("Percentage of redundant links: ", redundantLinks/len(edgelist))
    print("\n")
#     name = foodwebList[i]
    # To represent data in a dataframe
    AllFoodWebData.append(( len(allNodes), len(edgelist),len(spantreelinks),redundantLinks , redundantLinks/len(edgelist)))

######## Plot DFS tree with least connections ########
#     plt.figure()
#     plt.subplots(figsize=(30, 30))
#     plt.title(foodwebList[i], size=30)
#     nx.draw(spanningtree, with_labels=True)
#     plt.show()

# Tables and Charts

In [ ]:
## Organise data into data frame
# print(len(foodwebList))
foodwebListdf = pd.DataFrame(foodwebList)
foodwebListdf.columns = [("Food Web Name")]
AllFoodWebDatadf.columns = ["No. Nodes in Food Web","Number of Original edges", "Number of edges in spanning tree","Total Redundant links", "% of Redundant Links"]
Finaldf = pd.concat([foodwebListdf, AllFoodWebDatadf], axis=1)
Finaldf.head()


#Finaldf.to_csv("Spanning Tree Result.csv",  encoding='utf-8')

In [ ]:
Finaldf.plot(x ='Food Web Name', y='Total Redundant links', kind = 'bar',figsize=(40, 10))
plt.xticks(rotation=90)
plt.title('Redundant Links of All Food Webs', size=20)
plt.xlabel("Food Web Names", fontsize=18)
plt.ylabel("Total Redundant Links",fontsize=18)
plt.show()
#plt.savefig('redundant.png')

In [ ]:
Finaldf.plot(x ='Food Web Name', y='% of Redundant Links', kind = 'bar',figsize=(30, 5))
plt.xticks(rotation=90)
plt.title('% Redundant Links of All Food Webs', size=20)
plt.xlabel("Food Web Names", fontsize=18)
plt.ylabel("Total Redundant Links",fontsize=18)
plt.show()
#plt.savefig('redundant.png')

# Individual food web

In [ ]:
########## Sort consumer and resource column, find all node and edge list #########

### Specify Foodweb Here ###
Foodweb = df.loc[df['foodweb.name'] == 'Grand Caricaie  marsh dominated by Cladietum marisci, mown  Clmown1']

consumer = Foodweb['con.taxonomy']
resource = Foodweb['res.taxonomy']

allNodes = list(consumer) + list(resource)
allNodes = set(allNodes) # Removes duplicates
print(len(allNodes))


edgelistdf = Foodweb[['con.taxonomy','res.taxonomy']]
edgelist = list(map(tuple, edgelistdf.to_numpy()))
edgelist

In [ ]:
print(len(edgelist))

In [ ]:
#undirected graph
G = nx.Graph()

G.add_nodes_from(allNodes)
G.add_edges_from(edgelist)

print(nx.info(G))


In [ ]:
# undirected graph
plt.figure()
plt.subplots(figsize=(30, 30))
plt.title('Grand Caricaie  marsh dominated by Cladietum marisci, mown  Clmown1', size=30)
nx.draw_networkx(G, with_labels=True)
plt.show

In [ ]:
# Sort the node from highest to lowest degree
sortedNodes = sorted(G.degree, key = lambda x: x[1], reverse = True)

print(len(sortedNodes))

print(sortedNodes)

In [ ]:
# Sorting nodes in dictionary by its degree value to be used as look up
degree = {}
for i in sortedNodes:
    degree[i[0]] = i[1]
print(degree)

In [ ]:
###### Obstacles Found ######
# Counting how many upper degree nodes each node is connected to

# plotX = []
# for i in range(len(degree)):
#     plotX.append(i+1)
# print(plotX)

x = {}
degreeCounts = {}


for i in sortedNodes:                  # Goes through each nodes from highest to lowest degree
    count = 0 
    for j in edgelist:                 # If node is in edgelist connection 
        if i[0] in j:
            if j[0] == i[0]:
                if degree[j[1]] > degree[i[0]]: # Checks if the connected node has a higher degree
                    count = count +1
                    continue
            if j[1] == i[0]:
                if degree[j[0]] > degree[i[0]]: # Checks if the connected node has a higher degree
                    count = count + 1
                    continue
        degreeCounts[i[0]] = count
        
print(len(degreeCounts))
print(degreeCounts) #Emberiza schoeniclus has no higher degree nodes connected to it



In [ ]:
###### Obstacles Found ######
# Sorting dictionary from highest to lowest value

sorted_dict = {}
sorted_keys = sorted(degreeCounts, key=degreeCounts.get ,reverse = True)  

for w in sorted_keys:
    sorted_dict[w] = degreeCounts[w]

print(len(sorted_dict))
print(sorted_dict)

# Most resource species has multiple consumer species connected to it, some has only one. 
# Using DFS to simplify the network and find the resource species

# Depth First Search Spanning Tree (Obtain Resource Species)

In [ ]:
#Connects the network with the least number of links
spanningtree = nx.depth_first_search.dfs_tree(G,sortedNodes[0][0])

In [ ]:
edgelistst = list(spanningtree)
print(len(edgelistst))
#print(edgelistst)

In [ ]:
## Find the resource species using degree nodes (no of node adjacent to each node)

spanningTreeSortedNodes = sorted(spanningtree.degree, key = lambda x: x[1], reverse = True)
#print(spanningTreeSortedNodes)

resourceSpecies = []

for i in spanningTreeSortedNodes:
    
    if i[1] == 1:
        resourceSpecies.append(i)
        
#print(len(resourceSpecies))
#print("Resource Species", resourceSpecies)


In [ ]:
# Obtaining the resource species and remove apex species 

finalResourceSpecies = []

for i in resourceSpecies:
    if i[0]  != sortedNodes[0][0]:
        
        finalResourceSpecies.append(i[0])   
        
#print(len(finalResourceSpecies))
#print(finalResourceSpecies)

In [ ]:
plt.figure()
plt.subplots(figsize=(20, 20))
nx.draw(spanningtree, with_labels=True)
plt.show()

# Spanning tree loops

In [ ]:
SpanningTreeLists = []

redundantLinksCount = 0
numOfEdgesInSpanningTree = []

for i in finalResourceSpecies:                  # Goes through each resource species 
    
    spanningtree = []
    predatorList = []                           # List of predators for current basal species
    
    print("Trophic level 1 (Resource Species): ",i)
    
    if i not in spanningtree:      # Checks if root node is in the tree
        spanningtree.append(i)
        
    for j in edgelist:                 # If node is in edgelist connection  
        if j[1] == i:                    # Check if species is a resource species in edgelist
            if degree[j[0]] > degree[i]: # Checks if the connected node has a higher degree
                predatorList.append(j[0]) 
                
    if len(predatorList) == 0:                 # If length is zero print final spanning tree sequence
        print("Final Spanning Tree: ", spanningtree)
        SpanningTreeLists.append(spanningtree)
        TreeRedundancy(spanningtree)              # Adds final spanning tree edge for redundancy count
        continue
        
    pred = random.choice(predatorList)    # Select a random predator going up the food chain
    spanningtree.append(pred)
    
    print("Trophic level 2: ", predatorList)
##############################################################    
    predatorList2 = []                       # List of predators for predator species 

    for k in edgelist:                        # Loop through predator list     
        if k[1] == pred:
            if degree[k[0]] > degree[pred]:
                predatorList2.append(k[0])

    if len(predatorList2) == 0:               # If length is zero print final spanning tree sequence
        print("Final Spanning Tree: ", spanningtree)
        print("Redundant Links in this tree: ", len(predatorList)-1 )
        print("\n")
        redundantLinksCount = redundantLinksCount + len(predatorList)-1 
        #print("Total Redundant links in this foodweb: ",redundantLinksCount)
        SpanningTreeLists.append(spanningtree)
        TreeRedundancy(spanningtree)                  # Adds final spanning tree edge for redundancy count
        continue
        
    pred2 = random.choice(predatorList2)    # Select a random predator going up the food chain
    spanningtree.append(pred2)
    
   
        
    print("Trophic level 3: ", predatorList2)
##############################################################    
    predatorList3 = []                        # List of predators for predator species 

    for h in edgelist:                        # Loop through predator list     
        if h[1] == pred2:
            if degree[h[0]] > degree[pred2]:
                predatorList3.append(h[0])

    if len(predatorList3) == 0:                # If length is zero print final spanning tree sequence
        print("Final Spanning Tree: ", spanningtree)
        print("Redundant Links in this tree: ", len(predatorList) + len(predatorList2) -2 )
        print("\n")
        redundantLinksCount = redundantLinksCount + len(predatorList) + len(predatorList2) -2
        #print("Total Redundant links in this foodweb: ",redundantLinksCount)
        SpanningTreeLists.append(spanningtree)
        TreeRedundancy(spanningtree)                  # Adds final spanning tree edge for redundancy count
        continue
            
    pred3 = random.choice(predatorList3)    # Select a random predator going up the food chain
    spanningtree.append(pred3)
    

    
    print("Trophic level 4: ", predatorList3)
##############################################################    
    predatorList4 = []

    for h in edgelist:                        # Loop through predator list     
        if h[1] == pred3:
            if degree[h[0]] > degree[pred3]:
                predatorList3.append(h[0])
                
    if len(predatorList4) == 0:             # If length is zero print final spanning tree sequence
        print("Final Spanning Tree: ", spanningtree)
        print("Redundant Links in this tree: ", len(predatorList) + len(predatorList2) +len(predatorList3) -3 )
        print("\n")
        redundantLinksCount = redundantLinksCount + len(predatorList) + len(predatorList2) +len(predatorList3) -3
        #print("Total Redundant links in this foodweb: ",redundantLinksCount)
        SpanningTreeLists.append(spanningtree)
        TreeRedundancy(spanningtree)                  # Adds final spanning tree edge for redundancy count
        continue
        
    pred4 = random.choice(predatorList4)    # Select a random predator going up the food chain
    spanningtree.append(pred4)    
    
        
    print("Trophic level 5: ", predatorList4)
##############################################################    
    predatorList5 = []

    for h in edgelist:                        # Loop through predator list     
        if h[1] == pred4:
            if degree[h[0]] > degree[pred4]:
                predatorList4.append(h[0])
                
    if len(predatorList5) == 0:             # If length is zero print final spanning tree sequence
        print("Final Spanning Tree: ", spanningtree)
        print("Redundant Links in this tree:  \t", len(predatorList) + len(predatorList2) +len(predatorList3)\
                                         + len(predatorList4) -4 )
        print("\n")
        redundantLinksCount = redundantLinksCount +  len(predatorList) + len(predatorList2) +len(predatorList3)\
                                         + len(predatorList4) -4
        #print("Total Redundant links in this foodweb: ",redundantLinksCount)
        SpanningTreeLists.append(spanningtree)
        TreeRedundancy(spanningtree)                  # Adds final spanning tree edge for redundancy count
        continue
        
    pred5 = random.choice(predatorList5)    # Select a random predator going up the food chain
    spanningtree.append(pred5)    
            
    print("Trophic level 6: ", predatorList5)

spantreelinks = set(numOfEdgesInSpanningTree) # to remove same edge in spanning tree list
redundantLinks = len(edgelist) - len(spantreelinks)

print("Number of nodes in the food web: ", len(allNodes))
print("Original number of edges: ",len(edgelist))
print("Number of edges in spanning tree: ",len(spantreelinks))
print("Total Redundant links in this foodweb: ",redundantLinks)
print("Percentage of redundant links: ", redundantLinks/len(edgelist))
#print("Total Redundant links in this foodweb: ",redundantLinksCount)